## Imports

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import pandas as pd

import os

## Load the dataset

In [ ]:
folder = ".\\dataset\\walking"
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    if os.path.isfile(file_path):
        print(file_path)






.\dataset\walking\person01_walking_d1_uncomp.avi
.\dataset\walking\person01_walking_d2_uncomp.avi
.\dataset\walking\person01_walking_d3_uncomp.avi
.\dataset\walking\person01_walking_d4_uncomp.avi
.\dataset\walking\person02_walking_d1_uncomp.avi
.\dataset\walking\person02_walking_d2_uncomp.avi
.\dataset\walking\person02_walking_d3_uncomp.avi
.\dataset\walking\person02_walking_d4_uncomp.avi
.\dataset\walking\person03_walking_d1_uncomp.avi
.\dataset\walking\person03_walking_d2_uncomp.avi
.\dataset\walking\person03_walking_d3_uncomp.avi
.\dataset\walking\person03_walking_d4_uncomp.avi
.\dataset\walking\person04_walking_d1_uncomp.avi
.\dataset\walking\person04_walking_d2_uncomp.avi
.\dataset\walking\person04_walking_d3_uncomp.avi
.\dataset\walking\person04_walking_d4_uncomp.avi
.\dataset\walking\person05_walking_d1_uncomp.avi
.\dataset\walking\person05_walking_d2_uncomp.avi
.\dataset\walking\person05_walking_d3_uncomp.avi
.\dataset\walking\person05_walking_d4_uncomp.avi
.\dataset\walking\pe